Very broadly speaking, here's the pseudocode for a linear classification program implemented in tf.estimator:

In [1]:
import tensorflow as tf

# Set up a linear classifier.
classifier = tf.estimator.LinearClassifier(feature_columns)

# Train the model on some example data.
classifier.train(input_fn=train_input_fn, steps=2000)

# Use it to predict.
predictions = classifier.predict(input_fn=predict_input_fn)

C:\Users\bradburj\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


NameError: name 'feature_columns' is not defined

Load the necessary libraries

In [3]:
from __future__ import print_function

import math
from IPython import display
from matplotlib import cm
from matplotlib import gridspec
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import metrics
import tensorflow as tf
from tensorflow.python.data import Dataset

tf.logging.set_verbosity(tf.logging.ERROR)
pd.options.display.max_rows = 10
pd.options.display.float_format = '{:.1f}'.format

Next, we'll load our data set.

In [4]:
california_housing_dataframe = pd.read_csv("https://download.mlcc.google.com/mledu-datasets/california_housing_train.csv", sep=",")

Randomise the data, to be sure there we do not get any pathological ordering effetcs that might harm the performance of Stochastic Gradient Descent.
Additionally, we'll scale `median_house_value` to be in units of thousands, so it can be learned a little more easily with learning rates that are usually used.

In [6]:
california_housing_dataframe = california_housing_dataframe.reindex(
    np.random.permutation(california_housing_dataframe.index))
california_housing_dataframe["median_house_value"] /= 1000.0
california_housing_dataframe

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
2218,-117.4,34.6,18.0,755.0,169.0,483.0,165.0,1.4,0.1
9244,-119.1,35.4,20.0,892.0,129.0,331.0,135.0,7.2,0.2
384,-116.9,32.8,26.0,4129.0,714.0,1820.0,718.0,4.3,0.2
16294,-122.5,37.6,34.0,696.0,145.0,398.0,162.0,3.5,0.3
3867,-118.0,33.9,22.0,1432.0,335.0,746.0,296.0,2.0,0.1
...,...,...,...,...,...,...,...,...,...
1853,-117.3,33.0,21.0,2144.0,340.0,928.0,344.0,5.8,0.3
11942,-121.4,38.6,37.0,2839.0,390.0,1006.0,400.0,7.3,0.3
6919,-118.3,34.2,37.0,2144.0,446.0,860.0,435.0,3.9,0.3
11671,-121.3,38.7,13.0,2135.0,429.0,779.0,432.0,3.7,0.1


## Examine the Data
It's a good idea to get to know the data a little bit before working with it

In [7]:
california_housing_dataframe.describe()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
count,17000.0,17000.0,17000.0,17000.0,17000.0,17000.0,17000.0,17000.0,17000.0
mean,-119.6,35.6,28.6,2643.7,539.4,1429.6,501.2,3.9,0.2
std,2.0,2.1,12.6,2179.9,421.5,1147.9,384.5,1.9,0.1
min,-124.3,32.5,1.0,2.0,1.0,3.0,1.0,0.5,0.0
25%,-121.8,33.9,18.0,1462.0,297.0,790.0,282.0,2.6,0.1
50%,-118.5,34.2,29.0,2127.0,434.0,1167.0,409.0,3.5,0.2
75%,-118.0,37.7,37.0,3151.2,648.2,1721.0,605.2,4.8,0.3
max,-114.3,42.0,52.0,37937.0,6445.0,35682.0,6082.0,15.0,0.5


## Build the first model
In this exercise, we'll try to predict `median_house_value`, which will be our label. We'll u